In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd '/content/gdrive/MyDrive/VLSP2021-MRC'

/content/gdrive/.shortcut-targets-by-id/1-S62VF80vLD4deYL7ncHtff1BUGKdg56/VLSP2021-MRC


In [ ]:
!pip install transformers trankit

     |████████████████████████████████| 2.9 MB 4.1 MB/s 
     |████████████████████████████████| 773 kB 45.6 MB/s 
     |████████████████████████████████| 3.3 MB 35.6 MB/s 
     |████████████████████████████████| 895 kB 64.9 MB/s 
     |████████████████████████████████| 596 kB 88.8 MB/s 
     |████████████████████████████████| 56 kB 6.8 MB/s 
     |████████████████████████████████| 1.9 MB 57.0 MB/s 
     |████████████████████████████████| 776.8 MB 17 kB/s 
     |████████████████████████████████| 1.2 MB 59.1 MB/s 
  Created wheel for langid: filename=langid-1.1.6-py3-none-any.whl size=1941187 sha256=5cf207b02634d73363435aadfc8d79e0e91e1c34b4babbda49c37639eb609f7b
  Stored in directory: /root/.cache/pip/wheels/2b/bb/7f/11e4db39477278161e882eadc46fb558949a28b13470fc74b8
Successfully built langid
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: torch
    Found existin

In [ ]:
import string
import json
from trankit import Pipeline
from tqdm import tqdm

punctuation = string.punctuation + '…'

p = Pipeline('vietnamese', embedding='xlm-roberta-large')

Loading pretrained XLM-Roberta, this may take a while...
Loading tokenizer for vietnamese
Loading tagger for vietnamese
Loading lemmatizer for vietnamese
Loading NER tagger for vietnamese
Active language: vietnamese


In [ ]:
original_data = json.load(open('/content/gdrive/MyDrive/Data/DataVLSP/Private_data/private_test.json', 'r'))

for article in tqdm(original_data['data']):
    title = article['title']
    for paragraph in article['paragraphs']:
        context = paragraph['context']
        context = ' '.join(context.split())
        tokenized_context = p.tokenize(context)
        for qa in paragraph['qas']:
            question = qa['question']
            question = ' '.join(question.split())
            tokenized_question = p.tokenize(question, is_sent=True)
            qa['question'] = ' '.join([token['text'].replace(' ', '_') for token in tokenized_question['tokens']])
        paragraph['context'] = ' '.join([token['text'].replace(' ', '_') for sentence in tokenized_context['sentences'] for token in sentence['tokens']])


100%|██████████| 49/49 [16:48<00:00, 20.58s/it]


In [ ]:
for article in tqdm(original_data['data']):
    title = article['title']
    for paragraph in article['paragraphs']:
        context = paragraph['context']
        for qa in paragraph['qas']:
            question = qa['question']
            qa['question'] = question.replace('_', ' ')
        paragraph['context'] = context.replace('_', ' ')

100%|██████████| 49/49 [00:00<00:00, 7410.70it/s]


In [ ]:
def convert_test_format(original_data, file_out):
    data = original_data['data']

    for i in range(len(data)):
      for j in range(len(data[i]['paragraphs'])):
        for k in range(len(data[i]['paragraphs'][j]['qas'])):
          data[i]['paragraphs'][j]['qas'][k]['answers'] = []
          data[i]['paragraphs'][j]['qas'][k]['is_impossible'] = False

    public_test = {'version':'private test', "data": data}
    with open(file_out, 'w', encoding='utf-8') as outfile:
        json.dump(public_test, outfile ,ensure_ascii=False,indent=4)

convert_test_format(original_data,"/content/gdrive/MyDrive/Data/DataVLSP/Private_data/private_test_syllable.json")